# USSDFORGE

USSDforge is an AI-powered USSD code generation tool built for the African developer market. It leverages Anthropic's Claude to help developers rapidly build, convert, and validate USSD applications that conform to Nigerian telco standards (MTN, Airtel, Glo).

In [ ]:
# Install Dependencies
!pip install anthropic gradio

In [ ]:
# Imports, API Setup & Configuration
import os
import anthropic
import gradio as gr
import ast
import subprocess
import tempfile

# Load and validate API key
ANTHROPIC_API_KEY = os.environ.get("ANTHROPIC_API_KEY")

if not ANTHROPIC_API_KEY:
    raise EnvironmentError(
        "ANTHROPIC_API_KEY not found. Set it as an environment variable before running."
    )

# Initialize client and constants
client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)
MODEL = "claude-opus-4-6"
MAX_TOKENS = 8096



In [ ]:
#Core USSD Generation Logic


SUPPORTED_LANGUAGES = ["Python", "Node.js", "C++"]

SYSTEM_PROMPT = """You are an expert USSD developer specializing in African telco systems.
You generate clean, production-ready USSD session flow code following Nigerian telco standards (MTN, Airtel, Glo).

Rules:
- Use CON to continue the session and END to terminate it
- Always include input validation
- Structure code with clear session state handling
- Add brief comments only where necessary
- For C++ code, always include ALL necessary headers at the top (e.g. #include <iostream>, #include <string>, #include <vector>, #include <map>, #include <sstream>) before writing any code
- You MUST return raw code only
- Do NOT wrap code in markdown fences or backticks
- Do NOT include any explanation, preamble or summary before or after the code
- The very first character of your response must be the first character of the code"""


def generate_ussd_code(business_description, language):
    prompt = f"""Generate a USSD application in {language} for this business:

{business_description}

Return ONLY the raw {language} code. No markdown fences, no explanations, no preamble."""

    response = client.messages.create(
        model=MODEL,
        max_tokens=MAX_TOKENS,
        system=SYSTEM_PROMPT,
        messages=[{"role": "user", "content": prompt}]
    )
    return response.content[0].text.strip()


def convert_ussd_code(source_code, source_language, target_language):
    prompt = f"""Convert this USSD code from {source_language} to {target_language}.

{source_code}

Return ONLY the raw converted {target_language} code. No markdown fences, no explanations."""

    response = client.messages.create(
        model=MODEL,
        max_tokens=MAX_TOKENS,
        system=SYSTEM_PROMPT,
        messages=[{"role": "user", "content": prompt}]
    )
    return response.content[0].text.strip()


def validate_syntax(code, language):
    prompt = f"""You are a strict syntax validator. Analyze this {language} code for syntax errors only.

{code}

Rules:
- Check for syntax errors only, not logic or style issues
- If syntax is correct, respond with exactly: "Syntax check passed. No errors found."
- If there are errors, list each error with its line number and a brief description
- Do not suggest fixes, do not explain the code"""

    response = client.messages.create(
        model=MODEL,
        max_tokens=16000,
        system="You are a strict code syntax validator. Be precise and concise.",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.content[0].text.strip()




In [ ]:
# Gradio UI

copy_js = """
async (text) => {
    await navigator.clipboard.writeText(text);
    return text;
}
"""

def build_ui():
    with gr.Blocks(title="USSDforge") as app:
        gr.Markdown("# USSDforge\nGenerate and convert USSD flow code for African telco systems.")

        with gr.Tabs():

            with gr.Tab("Generate"):
                description_input = gr.Textbox(
                    label="Business Description",
                    placeholder="e.g. A savings app with balance check, deposit, and withdrawal menu",
                    lines=5
                )
                language_select = gr.Dropdown(
                    label="Output Language",
                    choices=SUPPORTED_LANGUAGES,
                    value="Python"
                )
                with gr.Row():
                    generate_btn = gr.Button("Generate Code", variant="primary")
                    copy_gen_btn = gr.Button("Copy Code", variant="secondary")
                generate_output = gr.Textbox(label="Generated USSD Code", lines=25, interactive=False)
                generate_btn.click(fn=generate_ussd_code, inputs=[description_input, language_select], outputs=generate_output)
                copy_gen_btn.click(fn=None, inputs=generate_output, outputs=generate_output, js=copy_js)

            with gr.Tab("Convert"):
                source_code_input = gr.Textbox(label="Source Code", lines=20)
                with gr.Row():
                    source_lang_select = gr.Dropdown(label="From", choices=SUPPORTED_LANGUAGES, value="Python")
                    target_lang_select = gr.Dropdown(label="To", choices=SUPPORTED_LANGUAGES, value="Node.js")
                with gr.Row():
                    convert_btn = gr.Button("Convert Code", variant="primary")
                    copy_conv_btn = gr.Button("Copy Code", variant="secondary")
                convert_output = gr.Textbox(label="Converted Code", lines=25, interactive=False)
                convert_btn.click(fn=convert_ussd_code, inputs=[source_code_input, source_lang_select, target_lang_select], outputs=convert_output)
                copy_conv_btn.click(fn=None, inputs=convert_output, outputs=convert_output, js=copy_js)

            with gr.Tab("Validate"):
                validate_code_input = gr.Textbox(label="Paste Code to Validate", lines=20)
                validate_lang_select = gr.Dropdown(label="Language", choices=SUPPORTED_LANGUAGES, value="Python")
                with gr.Row():
                    validate_btn = gr.Button("Check Syntax", variant="primary")
                    copy_val_btn = gr.Button("Copy Result", variant="secondary")
                validate_output = gr.Textbox(label="Validation Result", lines=4, interactive=False)
                validate_btn.click(fn=validate_syntax, inputs=[validate_code_input, validate_lang_select], outputs=validate_output)
                copy_val_btn.click(fn=None, inputs=validate_output, outputs=validate_output, js=copy_js)

    return app



In [ ]:
#Launch App


app = build_ui()
app.launch(
   
    share=True,
    inbrowser=True
   
)